In [1]:
import os
import pandas as pd
from qiime2 import Visualization
import matplotlib.pyplot as plt

%matplotlib inline

our_data = 'poop_data'

In [3]:
pwd

'/home/jovyan/Poop-Power'

In [4]:
! qiime tools peek $our_data/sequences.qza

UUID:        32a1795b-d6fb-4ecc-9166-4fe29fb8206a
Type:        SampleData[PairedEndSequencesWithQuality]
Data format: SingleLanePerSamplePairedEndFastqDirFmt


In [5]:
! qiime demux summarize \
    --i-data $our_data/sequences.qza \
    --o-visualization $our_data/sequences.qzv

Saved Visualization to: poop_data/sequences.qzv


In [6]:
Visualization.load(f'{our_data}/sequences.qzv')

<visualization: Visualization uuid: 8917018c-6dc1-4521-8080-687c6d715b63>

### Denoising - Amplicon Sequence Variants (ASV)
At first we want to denoise the data to create a feature table of ASVs. Our stept to apply Denoising (= remove noise) are the following):
1.Find the cleanest sequence (and/or model probabilities based on quality scores)
2.Correct and/or discard super noisy sequences (high expected errors)
3.DADA2: DADA2 builds an error model which can identify differences between sequences, filters out noisy sequences and generates a feature table with error-corrected sequences.


In [14]:
! qiime dada2 denoise-single \
    --i-demultiplexed-seqs $our_data/sequences.qza \
    --p-trunc-len 131 \   #we will truncate the reads up 131 bp (sequences longer than this have  smaller quality score).131 bp is our good enough, is it ok?
    --p-n-threads 3 \
    --o-table $our_data/dada2_table.qza \
    --o-representative-sequences $our_data/dada2_rep_set.qza \
    --o-denoising-stats $our_data/dada2_stats.qza

Saved FeatureTable[Frequency] to: poop_data/dada2_table.qza
Saved FeatureData[Sequence] to: poop_data/dada2_rep_set.qza
Saved SampleData[DADA2Stats] to: poop_data/dada2_stats.qza


In [15]:
! qiime metadata tabulate \
    --m-input-file $our_data/dada2_stats.qza \
    --o-visualization $our_data/dada2_stats.qzv

Saved Visualization to: poop_data/dada2_stats.qzv


In [16]:
Visualization.load(f'{our_data}/dada2_stats.qzv')

<visualization: Visualization uuid: 1650743d-18d8-4fa5-870e-d8c236d26550>

In [17]:
! qiime feature-table tabulate-seqs \
    --i-data $our_data/dada2_rep_set.qza \
    --o-visualization $our_data/dada2_rep_set.qzv

Saved Visualization to: poop_data/dada2_rep_set.qzv


In [18]:
Visualization.load(f'{our_data}/dada2_rep_set.qzv')

<visualization: Visualization uuid: 54e17101-7fae-47e2-9572-f34d24d427b9>

### Clustering - Operational Taxonomic Units (OUT)
With Clustering we are going to remove noisy sequences and reduce the amount of sequences to process. It works based on a given threshold, i.e. 97% similarity. Is less accurate then the denoising method.

In [19]:
! qiime quality-filter q-score --help

Usage: qiime quality-filter q-score [OPTIONS]

  This method filters sequence based on quality scores and the presence of
  ambiguous base calls.

Inputs:
  --i-demux ARTIFACT SampleData[SequencesWithQuality |
    PairedEndSequencesWithQuality]¹ | SampleData[JoinedSequencesWithQuality]²
                       The demultiplexed sequence data to be quality
                       filtered.                                    [required]
Parameters:
  --p-min-quality INTEGER
                       The minimum acceptable PHRED score. All PHRED scores
                       less that this value are considered to be low PHRED
                       scores.                                    [default: 4]
  --p-quality-window INTEGER
                       The maximum number of low PHRED scores that can be
                       observed in direct succession before truncating a
                       sequence read.                             [default: 3]
  --p-min-length-fraction NUMBER
        